In [1]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

## Generating Dataset  

In [2]:
data = make_blobs(n_samples=100000, n_features=12,centers=2,random_state=98)

features = data[0]
labels   = data[1]

#### Splitting the Dataset

Here we use function from sklearn Library to split our data into train dataset to train our model on it and test dataset to test our model after training

In [3]:
features_train,features_test,labels_train,labels_test = train_test_split(features,labels,test_size=0.30, random_state=1991)
labels_train = labels_train.reshape(1,len(labels_train))
labels_test = labels_test.reshape(1,len(labels_test))

#### Print the shape of train as well as test dataset 

In [4]:
print("Your training dataset consist of {} samples and {} features and the shape = ".format(features_train.shape[0],features_train.shape[1]), features_train.shape)
print("\nYour test dataset consist of {} samples and {} features and the shape = ".format(features_test.shape[0],features_test.shape[1]), features_test.shape)



Your training dataset consist of 70000 samples and 12 features and the shape =  (70000, 12)

Your test dataset consist of 30000 samples and 12 features and the shape =  (30000, 12)


# Try to build a neural network with n hidden layers, each containing different number of neurons and output Layer with single neuron
Activation Function : Sigmoid

Loss Function : Least Square Error 

## Exersise 1.A

##### Initialize the weight and bias for each Hidden Layer as well as output Layer 

In [5]:
np.random.seed(100)
# I will initialize the first Hidden Layer Weight & bias for You

n_hidden_layers  = 6

# n_hidden_neurons = [n_features,[n_neurons for each hidden_layer], n_output]
n_hidden_neurons = [features_train.shape[1],8,16,32,64,32,16,labels_train.shape[0]]  


#create dictionary to save all weights & biases in it
weight = dict()                         
bias   = dict() 
for i in range(n_hidden_layers+1):
    weight["weight_h"+str(i+1)] = np.random.randn(n_hidden_neurons[i+1],n_hidden_neurons[i])
    bias["bias_h"+str(i+1)]     = np.random.randn(n_hidden_neurons[i+1],1)
        
    

# you can call for Example weight_h1 from the dictionary as follow :
# weight["weight_h1"]

print(weight["weight_h1"].shape)

# the output_layer weight is "weight_h7"


(8, 12)


In [6]:
class activation_functions():
    
    def __init__(self):
        pass
    
    
    # activation functions (Forward)
    def sigmoid(self,z):
        
        return 1/(1+np.exp(-z))

    def softmax(self,z):
        shift_z = z - np.max(z)
        exps = np.exp(shift_z)
        a = exps / np.sum(exps, axis=0,keepdims=True)
        return a
    
    def relu(self,z):
        a = np.maximum(z,0)
        return a
    
    



## Exersise 1.B

Write a Class that contains 4 Functions ( sigmoid , loss function , forward_propagation, backward_propagation)


Least square Error = 
<img src="https://latex.codecogs.com/gif.latex?(1/n)&space;*&space;\sum&space;(labels&space;-&space;pred)^{2}" title="(1/n) * \sum (labels - pred)^{2}" />

n = number of samples ... in your case it's 700

In [7]:
class Deep_Neural_Network(activation_functions):
    
    def __init__(self,n_hidden_layers,weight,bias,learning_rate):
        
        super().__init__()
        self.n_hidden_layers = n_hidden_layers
        self.weight          = weight
        self.bias            = bias
        self.learning_rate   = learning_rate
        
    
    def activation_func(self,act_func,z):
        if act_func == "sigmoid":
            a = self.sigmoid(z)
            
        return a
    
    def cross_enthropy(self,labels_train,y_pred):
        
        loss = -labels_train*np.log(y_pred) - (1-labels_train)*np.log(1-y_pred)
        cost = np.mean(loss)
        
        return cost
    def least_square_error(self,labels_train,y_pred):
        
        return np.mean(np.square(labels_train-y_pred))
        
        
    def forward_propagation(self,features,labels):
        ### Here you will implement the feedforward of the Neural_Network
        act_func = ["sigmoid"]*(n_hidden_layers + 1) # you can change the activation function per layer if you want
                                                     # but here i will use sigmoid as activation function for all layers
        
        hidden_layer  = dict()
        activation    = dict()
        
        activation["features"] = features
        
        
        hidden_layer["hidden_layer_1"] = np.add(np.dot(self.weight["weight_h1"],features.T) , self.bias["bias_h1"]) 
        activation["activation_1"]     = self.activation_func(act_func[0],hidden_layer["hidden_layer_1"])
        
        for i in range(1,self.n_hidden_layers + 1):
            hidden_layer["hidden_layer_" + str(i+1)]  = np.add(np.dot(self.weight["weight_h" + str(i+1)],activation["activation_" + str(i)]) , self.bias["bias_h" + str(i+1)]) 
            activation["activation_" + str(i+1)]      = self.activation_func(act_func[i],hidden_layer["hidden_layer_" + str(i+1)])    
      
        
        y_pred = list(activation.values())[-1]   
        ### compute the loss
        
        loss = self.least_square_error(labels,y_pred)
        
        cache = (hidden_layer,activation)
        return loss,cache
        
        
        

    def backward_propagation(self,features,labels,cache):
        
        (hidden_layer,activation) = cache
        
        
        hidden_layer = list(hidden_layer.values())
        activation   = list(activation.values())
        n            = n_hidden_layers + 1
        dweight = dict()
        dbias   = dict()
        for j in range(len(hidden_layer)-1):
            if (j==0):
                dactivation = -2*(labels - activation[n-j])
            else:
                dactivation = np.dot(self.weight["weight_h" + str(n-(j-1))].T,dhidden_layer) 
                
            dhidden_layer                 = dactivation * (activation[n-j]*(1-activation[n-j]))
            dweight["dweight_h"+str(n-j)] = np.dot(dhidden_layer,activation[n -(j+1)].T)/features.shape[0] 
            dbias["dbias_h"+str(n-j)]     = np.mean(dhidden_layer,axis=1).reshape(-1,1)
            
            #update weight and bias
            
            self.weight["weight_h" + str(n-j)]  -= self.learning_rate*dweight["dweight_h"+str(n-j)]
            self.bias["bias_h" + str(n-j)]      -= self.learning_rate*dbias["dbias_h"+str(n-j)]

    

In [11]:
model = Deep_Neural_Network(n_hidden_layers,weight,bias,learning_rate=0.01)


In [12]:
# run this cell to train your model :D
epochs = 20
for epochs in range(epochs):
        loss, cache= model.forward_propagation(features_train,labels_train)
        model.backward_propagation(features,labels_train,cache)
        if (epochs % 10 == 0):
            print(loss)

0.28432653424204685
0.27633120072549583


# Here You can test your model and see tha accuracy :D

you just need to run both cells to see how good is your model and Good Luck 

In [13]:
def test(features_test,labels_test):
    
    loss, cache = model.forward_propagation(features_test,labels_test)
    
    y_pred = list(cache[1].values())[-1]  
                
    loss = model.least_square_error(labels_test,y_pred)
    print(loss)
    
    correct_answer = 0
    
    for i in range(features_test.shape[0]):
        if (labels_test[0][i] == np.round(y_pred[0][i])):
            correct_answer +=1
            
    
    accuracy = (correct_answer / features_test.shape[0]) * 100
    
    return accuracy
        
    

In [14]:
accuracy = test(features_test,labels_test)
print("The accuracy of your model is {}".format(accuracy))

0.2690727932141334
The accuracy of your model is 50.029999999999994
